In [79]:
import autograd.numpy as np
from autograd import grad


class Layer():
    def __init__(self, n0, n1, activation='ReLU', seed=42, leaky_alpha=0):
        self.w_shape = (n1, n0)
        self.activation = activation
        self.leaky_alpha = leaky_alpha
        self.seed = seed
    
    def output(self, input_vector, weight_mat, bias_vec): #Implements the computation at a single layer for 1 training example
        linear_combine = np.matmul(weight_mat, input_vector) + np.array(bias_vec) #Linearly combine and add bias
        
        output_vector = 0

        #Activate
        if self.activation == 'ReLU': output_vector = np.where(linear_combine>0, linear_combine, self.leaky_alpha * linear_combine)
        
        elif self.activation== 'Logistic': output_vector = 1 / (1  + np.exp(-1 * np.array(linear_combine)))
            
        
        elif self.activation == 'tanh': output_vector = np.tanh(linear_combine)
        elif self.activation == 'arctan': output_vector = np.arctan(linear_combine)
        elif self.activation == 'Softmax': output_vector = np.exp(linear_combine) / np.sum(np.exp(linear_combine))
        elif self.activation == 'None': output_vector = linear_combine
        
        return output_vector


    def __repr__(self):
        return '< ' + str(self.w_shape[1]) + ', ' + str(self.w_shape[0]) + ', ' + self.activation + ' >'

class NN():
    def __init__(self):
        self.layers = [] 
        
        self.num_params = 0
        self.allParams = []
        
    #Implementing the architecture
    def add_layer_at(self, layer, idx):
        self.layers.insert(idx, layer)
        
        if idx!=0 and idx!=len(self.layers)-1:
            if self.layers[idx-1].w_shape[0] != self.layers[idx].w_shape[1]:
                raise IndexError("Layer nodes must match")
            if self.layers[idx].w_shape[0] != self.layers[idx+1].w_shape[1]:
                raise IndexError("Layer nodes must match")
        elif idx==0:
            if self.layers[idx].w_shape[0] != self.layers[idx+1].w_shape[1]:
                raise IndexError("Layer nodes must match")
        else:
            if self.layers[idx-1].w_shape[0] != self.layers[idx].w_shape[1]:
                raise IndexError("Layer nodes must match")
        
        
    def add_layer(self, layer):
        self.layers.append(layer)
        if len(self.layers) >=2:
            if self.layers[-2].w_shape[0] != self.layers[-1].w_shape[1]:
                raise IndexError("Layer nodes must match")
        
    def remove_layer(self, layer_idx):
        del self.layers[layer_idx]
    
        
    def initialize(self): #Kaiming He initialization method –> initialize one long vector of all the NN's weights and biases
        self.num_params = sum([layer.w_shape[0] * (layer.w_shape[1] + 1) for layer in self.layers])
        self.allParams[:] = np.zeros(self.num_params)
        
        self.allParams.insert(0,0)
        
        border = 1
        for layer in self.layers:
            np.random.rand(layer.seed)
            params_len = layer.w_shape[0] * (layer.w_shape[1] + 1)
            self.allParams[border:border+params_len] = np.random.normal(0, 2/layer.w_shape[1], params_len)
            for i in range(border,border+params_len):
                if i%(layer.w_shape[1]+1) == 0 and i!=0: 
                    self.allParams[i] = 0.01
            border = border + params_len
        
             
    def forward_pass(self, input_vector): #1 forward pass for 1 training example
        layer_output = input_vector

        border = 1
        for layer in self.layers:
            np.random.seed(layer.seed)

            param_len = layer.w_shape[0] * (layer.w_shape[1] + 1)
            x = np.array(self.allParams[border: border+param_len]).reshape(1,-1)[0]
            params = x.reshape(layer.w_shape[0],layer.w_shape[1]+1)
            
            param_mat = params.reshape(layer.w_shape[0], layer.w_shape[1] + 1)
            
            weights = param_mat[:,:-1]
            biases = param_mat[:,-1]

            
            layer_output = layer.output(layer_output, weights, biases)

            border = border + param_len
            
        return layer_output


    def __repr__(self):
        return str(self.layers)


class Optimizer():
    def __init__(self, model, regularization='None', alpha=0, gamma=0):
        self.model = model
        self.regularization = regularization
        self.alpha = alpha
        self.gamma = gamma
    
    def train(self, allParams, train_features, targets, Loss, lr=0.01): #Performs 1 backprop step for 1 epoch
        if Loss == 'MSE':  
            gradient_fn = grad(self.MSE, 0)
        elif Loss == 'Cross Entropy Loss':
            gradient_fn = grad(self.Cross_Entropy_Loss, 0)
        else:
            raise ValueError("Enter a valid loss function")
                
        gradients = gradient_fn(allParams, train_features, targets)
        
        self.model.allParams -= np.array(lr * gradients)
        self.model.allParams = np.array([x._value for x in self.model.allParams])

        
        return gradients
    
    def Cross_Entropy_Loss(self, allParams, train_features, targets): #For classification tasks
        self.model.allParams[:] = allParams
        
        total_loss = 0
        for train_feature in train_features:
            prediction = self.model.forward_pass(train_feature)
            cost = np.sum( (-1 * np.log(prediction + 1e-8) * targets).flatten() )
            total_loss += cost
        avg_loss = (total_loss / len(train_features)) + self.decide_regularization(self.regularization, allParams)
        return avg_loss

    def MSE(self, allParams, train_features, targets): #For regression tasks
        self.model.allParams[:] = allParams
        
        total_loss = 0
        for i in enumerate(train_features):
            prediction = self.model.forward_pass(i[1])
            cost = (prediction - targets[[i[0]]])**2
            total_loss += cost
        mse = (total_loss / len(train_features)) + self.decide_regularization(self.regularization, allParams)
        return mse

    def decide_regularization(self, s, a): #Regularization options 
        if s == 'None':
            return 0
        elif s == 'Ridge':
            return self.alpha * np.sum(np.array( np.array(a)**2))
        elif s == 'Lasso':
            return self.alpha * np.sum(np.abs(np.array(a)))
        elif s == 'EN':
            return self.gamma * self.alpha * np.sum(np.array(a)**2) + (1-self.gamma) * self.alpha * np.sum(np.abs(np.array(a)))
        else:
            raise ValueError("Enter a valid regularization method")



In [151]:
from sklearn.datasets import load_digits

images = load_digits()['data'][:10] #an array of 100 flattened images, each image an array of length 64

labels = load_digits()['target'][:10] #an array of 100 corresponding labels (digits)
labels = np.eye(10)[labels] #One hot encoded (so that it can fit the dimensions of the model)


nn = NN()
nn.add_layer(Layer(64,50,'ReLU',42))
nn.add_layer(Layer(50,50,'Logistic',43))
nn.add_layer(Layer(50,50,'ReLU',44))
nn.add_layer(Layer(50,10,'Softmax',45))

nn.initialize()

opt = Optimizer(nn, 'Ridge', alpha=0.01)


opt.train(np.array(nn.allParams),images, labels, 'Cross Entropy Loss', lr=0.01)




























array([ 0.        ,  0.00946997, -0.00197198, ..., -0.00519003,
        0.03643563,  0.01278033])